In [27]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Each user is responsible for checking the content of datasets and the
# applicable licenses and determining if suitable for the intended use.

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_transformers4rec_getting-started-session-based-02-session-based-xlnet-with-pyt/nvidia_logo.png" style="width: 90px; float: right;">

# Session-based Recommendation with XLNET

This notebook is created using the latest stable [merlin-pytorch](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-pytorch/tags) container.

In this notebook we introduce the [Transformers4Rec](https://github.com/NVIDIA-Merlin/Transformers4Rec) library for sequential and session-based recommendation. This notebook uses the PyTorch API. Transformers4Rec integrates with the popular [HuggingFace’s Transformers](https://github.com/huggingface/transformers) and makes it possible to experiment with a cutting-edge implementation of the latest NLP Transformer architectures.  

We demonstrate how to build a session-based recommendation model with the [XLNET](https://arxiv.org/abs/1906.08237) Transformer architecture. The XLNet architecture was designed to leverage the best of both auto-regressive language modeling and auto-encoding with its Permutation Language Modeling training method. In this example we will use XLNET with masked language modeling (MLM) training method, which showed very promising results in the experiments conducted in our [ACM RecSys'21 paper](https://github.com/NVIDIA-Merlin/publications/blob/main/2021_acm_recsys_transformers4rec/recsys21_transformers4rec_paper.pdf).

In the previous notebook we went through our ETL pipeline with the NVTabular library, and created sequential features to be used in training a session-based recommendation model. In this notebook we will learn:

- Accelerating data loading of parquet files with multiple features on PyTorch using NVTabular library
- Training and evaluating a Transformer-based (XLNET-MLM) session-based recommendation model with multiple features

## Build a DL model with Transformers4Rec library  

Transformers4Rec supports multiple input features and provides configurable building blocks that can be easily combined for custom architectures:

- [TabularSequenceFeatures](https://nvidia-merlin.github.io/Transformers4Rec/stable/api/transformers4rec.torch.html#transformers4rec.torch.TabularSequenceFeatures) class that reads from schema and creates an input block. This input module combines different types of features (continuous, categorical & text) to a sequence.
-  [MaskSequence](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/stable/transformers4rec/torch/masking.py) to define masking schema and prepare the masked inputs and labels for the selected LM task.
-  [TransformerBlock](https://nvidia-merlin.github.io/Transformers4Rec/stable/api/transformers4rec.torch.html#transformers4rec.torch.TransformerBlock) class that supports HuggingFace Transformers for session-based and sequential-based recommendation models.
-  [SequentialBlock](https://nvidia-merlin.github.io/Transformers4Rec/stable/api/transformers4rec.torch.html#transformers4rec.torch.SequentialBlock) creates the body by mimicking [torch.nn.sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) class. It is designed to define our model as a sequence of layers.
-  [Head](https://nvidia-merlin.github.io/Transformers4Rec/stable/api/transformers4rec.torch.html#transformers4rec.torch.Head) where we define the prediction task of the model.
-  [NextItemPredictionTask](https://nvidia-merlin.github.io/Transformers4Rec/stable/api/transformers4rec.torch.html#transformers4rec.torch.NextItemPredictionTask) is the class to support next item prediction task.
- [Trainer](https://nvidia-merlin.github.io/Transformers4Rec/stable/api/transformers4rec.torch.html#transformers4rec.torch.Trainer) extends the `Trainer` class from HF transformers and manages the model training and evaluation.

You can check the [full documentation](https://nvidia-merlin.github.io/Transformers4Rec/) of Transformers4Rec if needed.

Figure 1 illustrates Transformers4Rec meta-architecture and how each module/block interacts with each other.

![tf4rec_meta](images/tf4rec_meta2.png)

### Import required libraries

In [1]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"]="0"

import glob
import torch 

from transformers4rec import torch as tr
from transformers4rec.torch.ranking_metric import NDCGAt, AvgPrecisionAt, RecallAt
from transformers4rec.torch.utils.examples_utils import wipe_memory

/opt/intel/oneapi/intelpython/latest/envs/transformer4rec/lib/python3.9/site-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")
/opt/intel/oneapi/intelpython/latest/envs/transformer4rec/lib/python3.9/site-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")
/opt/intel/oneapi/intelpython/latest/envs/transformer4rec/lib/python3.9/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


Transformers4Rec library relies on a schema object to automatically build all necessary layers to represent, normalize and aggregate input features. As you can see below, `schema.pb` is a protobuf file that contains metadata including statistics about features such as cardinality, min and max values and also tags features based on their characteristics and dtypes (e.g., categorical, continuous, list, integer).

### Set the schema object

We create the schema object by reading the processed train parquet file generated by NVTabular pipeline in the previous, `01-ETL-with-NVTabular`, notebook.

In [2]:
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "./data")
OUTPUT_DIR = os.environ.get("OUTPUT_DIR", f"{INPUT_DATA_DIR}/sessions_by_day")

In [12]:
from merlin.schema import Schema
from merlin.io import Dataset

train = Dataset(os.path.join(INPUT_DATA_DIR, "processed_nvt/part_0.parquet"))
schema = train.schema
schema

/opt/intel/oneapi/intelpython/latest/envs/transformer4rec/lib/python3.9/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


,name,tags,dtype,is_list,is_ragged,properties.embedding_sizes.dimension,properties.embedding_sizes.cardinality,properties.cat_path,properties.max_size,properties.freq_threshold,properties.num_buckets,properties.domain.min,properties.domain.max,properties.domain.name,properties.value_count.min,properties.value_count.max
0,session_id,(),"DType(name='int64', element_type=<ElementType....",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,day-first,(),"DType(name='int64', element_type=<ElementType....",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,item_id-list,"(Tags.CATEGORICAL, Tags.ID, Tags.LIST, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,52.0,507.0,.//categories/unique.item_id.parquet,0.0,0.0,NaN,0.0,506.0,item_id,2.0,14.0
3,category-list,"(Tags.CATEGORICAL, Tags.LIST)","DType(name='int64', element_type=<ElementType....",True,True,30.0,186.0,.//categories/unique.category.parquet,0.0,0.0,NaN,0.0,185.0,category,2.0,14.0
4,age_days-list,"(Tags.CONTINUOUS, Tags.LIST)","DType(name='float32', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,14.0
5,weekday_sin-list,"(Tags.CONTINUOUS, Tags.LIST)","DType(name='float32', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,14.0


In [28]:
train = Dataset(os.path.join(INPUT_DATA_DIR, "processed_nvt/part_0.parquet"))

/opt/intel/oneapi/intelpython/latest/envs/transformer4rec/lib/python3.9/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


In [29]:
import pandas as pd
data = pd.read_parquet(os.path.join("/home/vmagent/app/transformer4rec_recsys2022/data", "test_100.parquet"))

In [30]:
data

,item_id,y,session_id,feature,wf
0,"[6187, 19760, 2171, 15738, 21215, 17218]",21107,113,"[[0, 34, 78, 61, 23, 37, 25, 55, 30, 21, 92, 6...",0.574247
1,"[19705, 10442, 15969, 19705, 10442, 16631, 197...",25976,115,"[[0, 153, 115, 388, 26, 387, 110, 156, 322, 35...",0.926491
2,"[17428, 3946, 17428, 19143, 26673, 25002]",8840,261,"[[0, 145, 325, 50, 66, 28, 23, 1, 25, 326, 61,...",0.993254
3,"[19882, 4130, 4130, 11638, 9219]",4309,325,"[[0, 8, 313, 138, 83, 5, 86, 25, 35, 4, 39, 23...",0.571514
4,"[5264, 23345]",25415,332,"[[35, 8, 42, 28, 18, 33, 61, 39, 29, 27, 21, 3...",0.898270
...,...,...,...,...,...
95,"[23412, 17618, 15332, 21330]",23480,4684,"[[34, 0, 118, 21, 211, 27, 32, 92, 57, 279, 18...",0.973949
96,"[23742, 24881, 7244, 22339, 2640, 2640, 8335, ...",8861,4747,"[[0, 30, 216, 245, 134, 15, 29, 61, 14, 87, 23...",0.710444
97,"[21683, 17641, 7435, 17641, 4354, 24517, 18814...",23405,4964,"[[0, 314, 21, 19, 38, 15, 16, 371, 42, 13, 258...",0.850537
98,"[23345, 26402, 25320, 21939, 23345, 15777, 106...",23019,4976,"[[0, 136, 18, 55, 23, 378, 61, 21, 32, 29, 99,...",0.987960


In [31]:
from merlin.schema import Schema
from merlin.io import Dataset

train = Dataset(os.path.join("/home/vmagent/app/transformer4rec_recsys2022/data", "test_100.parquet"))
schema = train.schema

/opt/intel/oneapi/intelpython/latest/envs/transformer4rec/lib/python3.9/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


In [21]:
schema

,name,tags,dtype,is_list,is_ragged,properties.domain.min,properties.domain.max,properties.domain.name
0,item_id,"(Tags.ITEM, Tags.ID, Tags.CATEGORICAL, Tags.LIST)","DType(name='int64', element_type=<ElementType....",False,False,0.0,28143.0,item_id
1,y,"(Tags.TARGET, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,0.0,28143.0,y
2,session_id,(Tags.CONTINUOUS),"DType(name='int64', element_type=<ElementType....",False,False,0.0,4440001.0,session_id
3,feature,"(Tags.CATEGORICAL, Tags.LIST)","DType(name='int64', element_type=<ElementType....",False,False,0.0,903.0,feature
4,wf,(Tags.CONTINUOUS),"DType(name='float64', element_type=<ElementTyp...",False,False,NaN,NaN,NaN


In [13]:
# You can select a subset of features for training
schema = schema.select_by_name(['session_id','item_id-list', 
                                'category-list', 
                                'weekday_sin-list',
                                'age_days-list'])

Let's print out the schema.

In [14]:
schema

,name,tags,dtype,is_list,is_ragged,properties.embedding_sizes.dimension,properties.embedding_sizes.cardinality,properties.cat_path,properties.max_size,properties.freq_threshold,properties.num_buckets,properties.domain.min,properties.domain.max,properties.domain.name,properties.value_count.min,properties.value_count.max
0,session_id,(),"DType(name='int64', element_type=<ElementType....",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,item_id-list,"(Tags.CATEGORICAL, Tags.ID, Tags.LIST, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,52.0,507.0,.//categories/unique.item_id.parquet,0.0,0.0,NaN,0.0,506.0,item_id,2.0,14.0
2,category-list,"(Tags.CATEGORICAL, Tags.LIST)","DType(name='int64', element_type=<ElementType....",True,True,30.0,186.0,.//categories/unique.category.parquet,0.0,0.0,NaN,0.0,185.0,category,2.0,14.0
3,weekday_sin-list,"(Tags.CONTINUOUS, Tags.LIST)","DType(name='float32', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,14.0
4,age_days-list,"(Tags.CONTINUOUS, Tags.LIST)","DType(name='float32', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,14.0


### Define the sequential input module

Below we define our `input` block using the `TabularSequenceFeatures` [class](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/stable/transformers4rec/torch/features/sequence.py#L97). The `from_schema()` method processes the schema and creates the necessary layers to represent features and aggregate them. It keeps only features tagged as `categorical` and `continuous` and supports data aggregation methods like `concat` and `elementwise-sum`. It also supports data augmentation techniques like stochastic swap noise. It outputs an interaction representation after combining all features and also the input mask according to the training task (more on this later).


The `max_sequence_length` argument defines the maximum sequence length of our sequential input, and if `continuous_projection` argument is set, all numerical features are concatenated and projected by an MLP block so that continuous features are represented by a vector of size defined by user, which is `64` in this example.

In [15]:
inputs = tr.TabularSequenceFeatures.from_schema(
        schema,
        max_sequence_length=20,
        continuous_projection=64,
        masking="mlm",
        d_output=100,
)

The output of the `TabularSequenceFeatures` module is the sequence of interactions embedding vectors defined in the following steps:
- 1. Create sequence inputs: If the schema contains non sequential features, expand each feature to a sequence by repeating the value as many times as the `max_sequence_length` value.  
- 2. Get a representation vector of categorical features: Project each sequential categorical feature using the related embedding table. The resulting tensor is of shape (bs, max_sequence_length, embed_dim).
- 3. Project scalar values if `continuous_projection` is set : Apply an MLP layer with hidden size equal to `continuous_projection` vector size value. The resulting tensor is of shape (batch_size, max_sequence_length, continuous_projection).
- 4. Aggregate the list of features vectors to represent each interaction in the sequence with one vector: For example, `concat` will concat all vectors based on the last dimension `-1` and the resulting tensor will be of shape (batch_size, max_sequence_length, D) where D is the sum over all embedding dimensions and the value of continuous_projection. 
- 5. If masking schema is set (needed only for the `NextItemPredictionTask` training), the masked labels are derived from the sequence of raw item-ids and the sequence of interactions embeddings are processed to mask information about the masked positions.

### Define the Transformer block

In the next cell, the whole model is build with a few lines of code. 
Here is a brief explanation of the main classes:  
- [XLNetConfig](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/stable/transformers4rec/config/transformer.py#L261) - We have injected in the HF transformers config classes like `XLNetConfig`the `build()` method that provides default configuration to Transformer architectures for session-based recommendation. Here we use it to instantiate and configure an XLNET architecture.  
- [TransformerBlock](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/stable/transformers4rec/torch/block/transformer.py#L57) class integrates with HF Transformers, which are made available as a sequence processing module for session-based and sequential-based recommendation models.  
- [NextItemPredictionTask](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/stable/transformers4rec/torch/model/prediction_task.py#L110) supports the next-item prediction task. We also support other predictions [tasks](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/stable/transformers4rec/torch/model/prediction_task.py), like classification and regression for the whole sequence. 

In [16]:
# Define XLNetConfig class and set default parameters for HF XLNet config  
transformer_config = tr.XLNetConfig.build(
    d_model=64, n_head=4, n_layer=2, total_seq_length=20
)
# Define the model block including: inputs, masking, projection and transformer block.
body = tr.SequentialBlock(
    inputs, tr.MLPBlock([64]), tr.TransformerBlock(transformer_config, masking=inputs.masking)
)

# Define the evaluation top-N metrics and the cut-offs
metrics = [NDCGAt(top_ks=[20, 40], labels_onehot=True),  
           RecallAt(top_ks=[20, 40], labels_onehot=True)]

# Define a head related to next item prediction task 
head = tr.Head(
    body,
    tr.NextItemPredictionTask(weight_tying=True, 
                              metrics=metrics),
    inputs=inputs,
)

# Get the end-to-end Model class 
model = tr.Model(head)

Note that we can easily define an RNN-based model inside the `SequentialBlock` instead of a Transformer-based model. You can explore this [tutorial](https://github.com/NVIDIA-Merlin/Transformers4Rec/tree/stable/examples/tutorial) for a GRU-based model example.

### Train the model 

We use the Merlin Dataloader's PyTorch Dataloader for optimized loading of multiple features from input parquet files. You can learn more about this data loader [here](https://nvidia-merlin.github.io/dataloader).

### **Set Training arguments**

In [17]:
per_device_train_batch_size = int(os.environ.get(
    "per_device_train_batch_size", 
    '128'
))

per_device_eval_batch_size = int(os.environ.get(
    "per_device_eval_batch_size", 
    '32'
))

In [18]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer
# Set hyperparameters for training 
train_args = T4RecTrainingArguments(data_loader_engine='merlin', 
                                    dataloader_drop_last = True,
                                    gradient_accumulation_steps = 1,
                                    per_device_train_batch_size = per_device_train_batch_size, 
                                    per_device_eval_batch_size = per_device_eval_batch_size,
                                    output_dir = "./tmp", 
                                    learning_rate=0.0005,
                                    lr_scheduler_type='cosine', 
                                    learning_rate_num_cosine_cycles_by_epoch=1.5,
                                    num_train_epochs=5,
                                    max_sequence_length=20, 
                                    report_to = [],
                                    logging_steps=50,
                                    no_cuda=True)

Note that we add an argument `data_loader_engine='merlin'` to automatically load the features needed for training using the schema. The default value is `merlin` for optimized GPU-based data-loading. Optionally a `PyarrowDataLoader` (pyarrow) can also be used as a basic option, but it is slower and works only for small datasets, as the full data is loaded to CPU memory.

## Daily Fine-Tuning: Training over a time window

Here we do daily fine-tuning meaning that we use the first day to train and second day to evaluate, then we use the second day data to train the model by resuming from the first step, and evaluate on the third day, so on and so forth.

We have extended the HuggingFace transformers `Trainer` class (PyTorch only) to support evaluation of RecSys metrics. In this example, the evaluation of the session-based recommendation model is performed using traditional Top-N ranking metrics such as Normalized Discounted Cumulative Gain (NDCG@20) and Hit Rate (HR@20). NDCG accounts for rank of the relevant item in the recommendation list and is a more fine-grained metric than HR, which only verifies whether the relevant item is among the top-n items. HR@n is equivalent to Recall@n when there is only one relevant item in the recommendation list.

In [19]:
# Instantiate the T4Rec Trainer, which manages training and evaluation for the PyTorch API
trainer = Trainer(
    model=model,
    args=train_args,
    schema=schema,
    compute_metrics=True,
)

/opt/intel/oneapi/intelpython/latest/envs/transformer4rec/lib/python3.9/site-packages/accelerate/utils/imports.py:265: UserWarning: Intel Extension for PyTorch 1.12 needs to work with PyTorch 1.12.*, but PyTorch 2.0.1 is found. Please switch to the matching version and run again.
  warnings.warn(
Detected kernel version 5.3.11, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [20]:
train_paths = glob.glob(os.path.join(INPUT_DATA_DIR, "processed_nvt/part_0.parquet"))

trainer.train_dataset_or_path = train_paths
trainer.reset_lr_scheduler()
trainer.train()
trainer.state.global_step +=1
print('finished')

wipe_memory()

/opt/intel/oneapi/intelpython/latest/envs/transformer4rec/lib/python3.9/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/opt/intel/oneapi/intelpython/latest/envs/transformer4rec/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
50,5.681000
100,5.241800



KeyboardInterrupt



Define the output folder of the processed parquet files:

In [38]:
start_window_index = int(os.environ.get(
    "start_window_index", 
    '1'
))

final_window_index = int(os.environ.get(
    "final_window_index", 
    '8'
))

In [39]:
start_time_window_index = start_window_index
final_time_window_index = final_window_index
#Iterating over days of one week
for time_index in range(start_time_window_index, final_time_window_index):
    # Set data 
    time_index_train = time_index
    time_index_eval = time_index + 1
    train_paths = glob.glob(os.path.join(OUTPUT_DIR, f"{time_index_train}/train.parquet"))
    eval_paths = glob.glob(os.path.join(OUTPUT_DIR, f"{time_index_eval}/valid.parquet"))
    print(train_paths)
    
    # Train on day related to time_index 
    print('*'*20)
    print("Launch training for day %s are:" %time_index)
    print('*'*20 + '\n')
    trainer.train_dataset_or_path = train_paths
    trainer.reset_lr_scheduler()
    trainer.train()
    trainer.state.global_step +=1
    print('finished')
    
    # Evaluate on the following day
    trainer.eval_dataset_or_path = eval_paths
    train_metrics = trainer.evaluate(metric_key_prefix='eval')
    print('*'*20)
    print("Eval results for day %s are:\t" %time_index_eval)
    print('\n' + '*'*20 + '\n')
    for key in sorted(train_metrics.keys()):
        print(" %s = %s" % (key, str(train_metrics[key]))) 
    wipe_memory()

/venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1664
  Num Epochs = 5
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 65


['/workspace/data/sessions_by_day/1/train.parquet']
********************
Launch training for day 1 are:
********************



Step,Training Loss
50,5.878700




Training completed. Do not forget to share your model on huggingface.co/models =)




finished


...

finished
********************
Eval results for day 7 are:	

********************

 eval_/loss = 4.442317962646484
 eval_/next-item/ndcg_at_20 = 0.1833297461271286
 eval_/next-item/ndcg_at_40 = 0.23786288499832153
 eval_/next-item/recall_at_20 = 0.5052083730697632
 eval_/next-item/recall_at_40 = 0.7760416865348816
 eval_runtime = 0.143
 eval_samples_per_second = 1342.609
 eval_steps_per_second = 41.957
['/workspace/data/sessions_by_day/7/train.parquet']
********************
Launch training for day 7 are:
********************



***** Running training *****
  Num examples = 1792
  Num Epochs = 5
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 70


Step,Training Loss
50,4.489300




Training completed. Do not forget to share your model on huggingface.co/models =)




finished
********************
Eval results for day 8 are:	

********************

 eval_/loss = 4.538415431976318
 eval_/next-item/ndcg_at_20 = 0.17900079488754272
 eval_/next-item/ndcg_at_40 = 0.22817844152450562
 eval_/next-item/recall_at_20 = 0.5052083730697632
 eval_/next-item/recall_at_40 = 0.7447916865348816
 eval_runtime = 0.1381
 eval_samples_per_second = 1390.084
 eval_steps_per_second = 43.44


### Re-compute evaluation metrics of the validation data

In [40]:
eval_data_paths = glob.glob(os.path.join(OUTPUT_DIR, f"{time_index_eval}/valid.parquet"))

In [41]:
# set new data from day 7
eval_metrics = trainer.evaluate(eval_dataset=eval_data_paths, metric_key_prefix='eval')
for key in sorted(eval_metrics.keys()):
    print("  %s = %s" % (key, str(eval_metrics[key])))

  eval_/loss = 4.538415431976318
  eval_/next-item/ndcg_at_20 = 0.17900079488754272
  eval_/next-item/ndcg_at_40 = 0.22817844152450562
  eval_/next-item/recall_at_20 = 0.5052083730697632
  eval_/next-item/recall_at_40 = 0.7447916865348816
  eval_runtime = 0.1281
  eval_samples_per_second = 1498.788
  eval_steps_per_second = 46.837


### Save the model

Let's save the model to be able to load it back at inference step. Using `model.save()`, we save the model as a pkl file in the given path.

In [42]:
model_path= os.environ.get("OUTPUT_DIR", f"{INPUT_DATA_DIR}/saved_model")
model.save(model_path)

That's it! You have just trained your session-based recommendation model using Transformers4Rec. Now you can move on to the next notebook `03-serving-session-based-model-torch-backend`. Please shut down this kernel to free the GPU memory before you start the next one.

Tip: We can easily log and visualize model training and evaluation on [Weights & Biases (W&B)](https://wandb.ai/home), [TensorBoard](https://www.tensorflow.org/tensorboard), or [NVIDIA DLLogger](https://github.com/NVIDIA/dllogger). By default, the HuggingFace transformers `Trainer` (which we extend) uses Weights & Biases (W&B) to log training and evaluation metrics, which provides nice visualization results and comparison between different runs.